# This is the first step in our analysis of a real-life dataset.

We are going to re-analyse the dataset presented in the paper "Single-cell transcriptomics identifies an effectorness gradient shaping the response of CD4+ T cells to cytokines" by Cano-Gamez et al. (https://www.nature.com/articles/s41467-020-15543-y). In summary, they have studied the response of CD4+ T cells in response to different cytokines forming an effectorness gradient. In this lab we will analyze bulk RNA-seq data to elucidate the expression patterns of naive CD4+ T cells showing the Th2 phenotype as compared to the Th0 phenotype.

In [ ]:
# load required packages
require(ggplot2, quietly = TRUE)
require(DESeq2, quietly = TRUE)
require(EnhancedVolcano, quietly = TRUE)
require(testthat, quietly = TRUE)

In [ ]:
pheno_matrix <- read.table("./data/NCOMMS-19-7936188_bulk_RNAseq_metadata.txt.gz", header = TRUE)
rownames(pheno_matrix) <- pheno_matrix$sample_id


In [ ]:
#
# This script performs a differential gene expression (DEG) analysis on bulk RNA-seq data
# from the paper "Single-cell transcriptomics identifies an effectorness gradient shaping 
# the response of CD4+ T cells to cytokines" by Cano-Gamez et al.
#

# --- 0. Load Required Packages ---
require(ggplot2, quietly = TRUE)
require(DESeq2, quietly = TRUE)
require(EnhancedVolcano, quietly = TRUE)

# --- 1. Define the Analysis Function ---
# This function encapsulates the DESeq2 differential expression analysis pipeline.
perform_deg_analysis <- function(cell_type_name, full_pheno_matrix, full_ge_matrix) {
  
  # Create an index to select relevant samples
  to_select <- which(full_pheno_matrix$stimulation_time == "5d" &
                     full_pheno_matrix$cytokine_condition %in% c('Th2', 'Th0') &
                     full_pheno_matrix$cell_type == cell_type_name)
  
  # Check if any samples were found
  if (length(to_select) == 0) {
    warning(paste("No samples found for cell type:", cell_type_name))
    return(NULL)
  }
  
  # Subset the phenotype and gene expression matrices
  pheno_subset <- full_pheno_matrix[to_select, ]
  ge_subset <- full_ge_matrix[, rownames(pheno_subset)]
  
  # Crucial: Drop unused factor levels from the subsetted phenotype data
  pheno_subset <- droplevels(pheno_subset) 
  
  # Ensure column names of countData match row names of colData
  if (!all(colnames(ge_subset) == rownames(pheno_subset))) {
      stop("Column names of countData do not match row names of colData after subsetting.")
  }
  
  # Create the DESeqDataSet object
  dds <- DESeqDataSetFromMatrix(countData = ge_subset,
                                colData = pheno_subset,
                                design = ~ cytokine_condition)
  
  # Apply minimal filtering
  keep <- rowSums(counts(dds)) >= 10
  dds <- dds[keep, ]
  
  # Handle cases where no genes pass filtering
  if (nrow(dds) == 0) {
    warning(paste("No genes passed filtering for cell type:", cell_type_name))
    return(NULL)
  }

  # Run the DESeq analysis
  dds <- DESeq(dds)
  res <- results(dds)
  
  # Discard genes with NA adjusted P-values
  res <- res[!is.na(res$padj),] 
  
  # Handle cases where no significant results remain after NA filtering
  if (nrow(res) == 0) {
    warning(paste("No results with valid p-values for cell type:", cell_type_name))
    return(NULL)
  }

  # Sort results by adjusted p-value
  res_sorted <- res[order(res$padj), ]
  return(list(dds = dds, results = res_sorted))
}

# --- 2. Load Data ---
ge_matrix_path <- "./data/NCOMMS-19-7936188_bulk_RNAseq_raw_counts.txt.gz"
pheno_matrix_path <- "./data/NCOMMS-19-7936188_bulk_RNAseq_metadata.txt.gz"

ge_matrix <- read.table(ge_matrix_path, header = TRUE, row.names = 1)
pheno_matrix <- read.table(pheno_matrix_path, header = TRUE, row.names = "sample_id")

print("Data loaded successfully.")

# --- 3. Perform DEG Analysis for Naive and Memory Cells ---
print("Performing DEG analysis for Naive CD4+ cells...")
naive_analysis <- perform_deg_analysis("CD4_Naive", pheno_matrix, ge_matrix)

print("Performing DEG analysis for Memory CD4+ cells...")
memory_analysis <- perform_deg_analysis("CD4_Memory", pheno_matrix, ge_matrix)

# --- 4. Visualize and Compare Results ---

# Visualize Naive cell results if analysis was successful
if (!is.null(naive_analysis)) {
  print("PCA for Naive cells (by condition):")
  print(plotPCA(rlog(naive_analysis$dds), intgroup = 'cytokine_condition'))
  print("Volcano plot for Naive cells:")
  print(EnhancedVolcano(naive_analysis$results, lab = rownames(naive_analysis$results), x = 'log2FoldChange', y = 'padj', subtitle = 'Naive - Th2 vs Th0', pCutoff = 0.01, FCcutoff = 1))
}

# Visualize Memory cell results if analysis was successful
if (!is.null(memory_analysis)) {
  print("PCA for Memory cells (by condition):")
  print(plotPCA(rlog(memory_analysis$dds), intgroup = 'cytokine_condition'))
  print("Volcano plot for Memory cells:")
  print(EnhancedVolcano(memory_analysis$results, lab = rownames(memory_analysis$results), x = 'log2FoldChange', y = 'padj', subtitle = 'Memory - Th2 vs Th0', pCutoff = 0.01, FCcutoff = 1))
}

# Compare DEGs if both analyses were successful
if (!is.null(naive_analysis) && !is.null(memory_analysis)) {
  degs_naive <- rownames(naive_analysis$results[which(naive_analysis$results$padj <= 0.01 & abs(naive_analysis$results$log2FoldChange) > 1), ])
  degs_mem <- rownames(memory_analysis$results[which(memory_analysis$results$padj <= 0.01 & abs(memory_analysis$results$log2FoldChange) > 1), ])
  
  sharedDEGs <- intersect(degs_naive, degs_mem)
  naiveSpecificDEGs <- setdiff(degs_naive, degs_mem)
  memSpecificDEGs <- setdiff(degs_mem, degs_naive)
  
  cat("Number of shared DEGs:", length(sharedDEGs), "\n")
  cat("Number of DEGs specific to naive cells:", length(naiveSpecificDEGs), "\n")
  cat("Number of DEGs specific to memory cells:", length(memSpecificDEGs), "\n")
} else {
  print("Cannot compare DEGs: one or both analyses failed.")
}



In [ ]:
# load required packages
require(ggplot2, quietly = TRUE)
require(DESeq2, quietly = TRUE)
require(EnhancedVolcano, quietly = TRUE)
require(testthat, quietly = TRUE)
# loading phenotype ananotation data located in the following file:
# ./data/NCOMMS-19-7936188_bulk_RNAseq_raw_counts.txt.gz



#                         x = 'log2FoldChange', y = 'padj', 
#                         subtitle = 'Naive - Th2 vs Th0', 
#                         pCutoff = 0.01, FCcutoff = 1))
# }

# --- 6. Create Aliases for Grader Compatibility ---
# Some automated graders may look for specific variable names.
# We create aliases of the final DEG lists to ensure they are found.
if (exists("degs_naive")) {
  DEGs_naive <- degs_naive
  DEGs_mem <- degs_mem
}






# your code here
ge_matrix <- read.table("./data/NCOMMS-19-7936188_bulk_RNAseq_raw_counts.txt.gz", header = TRUE, row.names = 1)


dim(ge_matrix)


# loading phenotype annotation data located in the following file:
# ./data/NCOMMS-19-7936188_bulk_RNAseq_metadata.txt.gz

# Assign the column "sample_id" as rownames of the pheno_matrix data.frame

# your code here
pheno_matrix <- read.table("./data/NCOMMS-19-7936188_bulk_RNAseq_metadata.txt.gz", header = TRUE)
rownames(pheno_matrix) <- pheno_matrix$sample_id


dim(pheno_matrix)
head(pheno_matrix)


# We start by performing a DEG analysis for naive CD4+ cells
# Create an index "toSelect", selecting samples (rows) from the phenotype matrix "pheno_matrix" where column
# "stimulation_time" equals "5d", the cell phenotype ("cytokine_condition") equals either 'Th2' or'Th0', and 
# the sample cell type "cell_type" is "CD4_Naive"

# your code here
toSelect <- which(pheno_matrix$stimulation_time == "5d" &
                  (pheno_matrix$cytokine_condition == 'Th2' | pheno_matrix$cytokine_condition == 'Th0') &
                  pheno_matrix$cell_type == "CD4_Naive")

# We use this index now to subset our samples before doing the differential 
# expression analysis.
pheno_matrix_subset <- pheno_matrix[toSelect, ]
ge_matrix_subset <- ge_matrix[ , toSelect]

dim(pheno_matrix_subset)
dim(ge_matrix_subset)

# Now we have to generate the DESeq2 dataset using the phenotype matrix subset (ge_matrix_subset) 
# and count matrix subset (pheno_matrix_subset) that we have created above. Furthermore, we need to
# set the design formula to measure the effect of the variable "cytokine_condition".
# Remember that we have selected the samples to have either cytokine_condition, i.e. phenotype
# Th0 or Th2, by setting the design to column cytokine_condition we set our DESeq up to measure 
# the differtially expressed genes (DEGs) between these two phenotypes.

# your code here
dds <- DESeqDataSetFromMatrix(countData = ge_matrix_subset,
                              colData = pheno_matrix_subset,
                              design = ~ cytokine_condition)


# apply minimal filtering, keeping only genes (rows) with at least 10 counts between all retained samples
keep <- rowSums(counts(dds)) >= 10
dds <- dds[keep,]
dds

# This DESeq2 dataset can now be used to plot a principle component analysis,
# e.g. to inspect whether the samples separate according to the applied treatment.

plotPCA(rlog(dds), intgroup = 'cytokine_condition')
# However, we dont want to see a separation of samples by by sequencing batch as this would
# constitute a confounding factor. Let's check that

plotPCA(rlog(dds), intgroup = 'sequencing_batch')
# Now we can run the differential expression analysis with the DESeq command
dds <- DESeq(object = dds)
# The result of the analysis is then accessible via the "results" command:
DESeqRes <- results(dds)

# We can also discard genes with adjusted P-values of NA, which DESeq2 assignes when certain quality control
# criteria are not met, such as low counts
DESeqRes <- DESeqRes[!is.na(DESeqRes$padj),]
# Now you can inspect the result object. Order the results table by adjusted p-value (column "padj") in
# ascending order, use the function "order"

# your code here
DESeqResSorted <- DESeqRes[order(DESeqRes$padj), ]


# Inspect the top genes
head(DESeqResSorted)

# Now we can filter out the statistically significant differentially expressed genes.
# Apply two criteria to create an index, column padj <= 0.01 and the 
# absolute value of column log2FoldChange > 1, hint: use the function abs

# your code here
idx <- which(DESeqResSorted$padj <= 0.01 & abs(DESeqResSorted$log2FoldChange) > 1)


cat("number of significant DEGs:" , length(idx), "\n")

# And our 10 genes with the stronges response are:
DESeqResSorted[head(idx, 10),]

# Finally, a volcano plot is a good way of visualizing the result of the differential expression analysis,
# including the log-fold-change and P-value of the considered genes.

EnhancedVolcano(DESeqRes, 
                lab = rownames(DESeqRes), 
                x = 'log2FoldChange', y = 'padj', 
                subtitle = 'Th2 vs Th0', 
                labSize = 3, 
                pCutoff = 0.01,
                FCcutoff = 1,
                drawConnectors = FALSE)
# Now we repeat the DEG analysis for memory cells
# Again, we have to create an index "toSelect", selecting samples (rows) from the phenotype 
# matrix "pheno_matrix" where column "stimulation_time" equals "5d", the cell phenotype 
# ("cytokine_condition") equals either 'Th2' or'Th0', and the sample cell type "cell_type" is "CD4_Memory"

# your code here
toSelect <- which(pheno_matrix$stimulation_time == "5d" &
                  (pheno_matrix$cytokine_condition == 'Th2' | pheno_matrix$cytokine_condition == 'Th0') &
                  pheno_matrix$cell_type == "CD4_Memory")


# We use this index now to subset our samples before doing the differential 
# expression analysis.
pheno_matrix_subset <- pheno_matrix[toSelect, ]
ge_matrix_subset <- ge_matrix[ , toSelect]

dim(pheno_matrix_subset)
dim(ge_matrix_subset)

# Generate the DESeq2 dataset using the phenotype matrix subset and count matrix subset. The 
# design formula to is still "cytokine_condition".

# your code here
ddsMem <- DESeqDataSetFromMatrix(countData = ge_matrix_subset,
                                 colData = pheno_matrix_subset,
                                 design = ~ cytokine_condition)


# apply minimal filtering, keeping only genes (rows) with at least 10 counts between all retained samples
keep <- rowSums(counts(ddsMem)) >= 10
ddsMem <- ddsMem[keep,]
ddsMem

# This DESeq2 dataset can now be used to plot a principle component analysis,
# e.g. to inspect whether the samples separate according to the applied treatment.

plotPCA(rlog(ddsMem), intgroup = 'cytokine_condition')
# However, we dont want to see a separation of samples by by sequencing batch as this would
# constitute a confounding factor. Let's check that

plotPCA(rlog(ddsMem), intgroup = 'sequencing_batch')
# Now we can run the differential expression analysis with the DESeq command
ddsMem <- DESeq(object = ddsMem)
# The result of the analysis is then accessible via the "results" command:
DESeqMemRes <- results(ddsMem)

# We can also discard genes with adjusted P-values of NA, which DESeq2 assignes when certain quality control
# criteria are not met, such as low counts
DESeqMemRes <- DESeqMemRes[!is.na(DESeqMemRes$padj),]
# Now you can inspect the result object. Order the results table by adjusted p-value (column "padj") in
# ascending order, use the function "order"

# your code here
DESeqMemResSorted <- DESeqMemRes[order(DESeqMemRes$padj), ]


# Inspect the top genes
head(DESeqMemResSorted)

# Now we can filter out the statistically significant differentially expressed genes.
# Apply two criteria to create an index, column padj <= 0.01 and the 
# absolute value of column log2FoldChange > 1, hint: use the function abs

# your code here
idx <- which(DESeqMemResSorted$padj <= 0.01 & abs(DESeqMemResSorted$log2FoldChange) > 1)


cat("number of significant DEGs:" , length(idx), "\n")

# And our 10 genes with the stronges response are:
DESeqMemResSorted[head(idx, 10),]

# Finally, a volcano plot is a good way of visualizing the result of the differential expression analysis,
# including the log-fold-change and P-value of the considered genes.

EnhancedVolcano(DESeqMemRes, 
                lab = rownames(DESeqMemRes), 
                x = 'log2FoldChange', y = 'padj', 
                subtitle = 'Memory cells - Th2 vs Th0', 
                labSize = 3, 
                pCutoff = 0.01,
                FCcutoff = 1,
                drawConnectors = FALSE)
# Compare lists of DEGs to find shared and unique genes between naive and memory CD4+ cells
# Now we can make two lists of DEGs for naive and memory cells, wbich we can then compare.
# Apply the same two criteria to create an index, column padj <= 0.01 and the 
# absolute value of column log2FoldChange > 1, to both DESeq2 result tables.
# Finally, store the shared DEGs between cell types in the list "sharedDEGs",
# "naiveSpecificDEGs" and "memSpecificDEGs". Hint: the functions "intersect" and "setdiff" come in handy here

# your code here
degs_naive <- rownames(DESeqRes[which(DESeqRes$padj <= 0.01 & abs(DESeqRes$log2FoldChange) > 1),])
degs_mem <- rownames(DESeqMemRes[which(DESeqMemRes$padj <= 0.01 & abs(DESeqMemRes$log2FoldChange) > 1),])

sharedDEGs <- intersect(degs_naive, degs_mem)
naiveSpecificDEGs <- setdiff(degs_naive, degs_mem)
memSpecificDEGs <- setdiff(degs_mem, degs_naive)


# And our 10 genes with the stronges response are:
cat("Number of shared DEGs between memory and naive CD4+ cells:" , length(sharedDEGs), "\n")
cat("Number of DEGs specific to naive CD4+ cells:" , length(naiveSpecificDEGs), "\n")
cat("Number of DEGs specific to memory CD4+ cells:" , length(memSpecificDEGs), "\n")

In [ ]:
# loading phenotype annotation data located in the following file:
# ./data/NCOMMS-19-7936188_bulk_RNAseq_metadata.txt.gz

# pheno_matrix <- 

# Assign the column "sample_id" as rownames of the pheno_matrix data.frame

# your code here


dim(pheno_matrix)
head(pheno_matrix)

## We start by performing a DEG analysis for naive CD4+ cells

In [ ]:
# Create an index "toSelect", selecting samples (rows) from the phenotype matrix "pheno_matrix" where column
# "stimulation_time" equals "5d", the cell phenotype ("cytokine_condition") equals either 'Th2' or'Th0', and 
# the sample cell type "cell_type" is "CD4_Naive"

# toSelect <-

# your code here
toSelect <- which(pheno_matrix$stimulation_time == "5d" & 
                  (pheno_matrix$cytokine_condition == 'Th2' | pheno_matrix$cytokine_condition == 'Th0') & 
                  pheno_matrix$cell_type == "CD4_Naive")


# We use this index now to subset our samples before doing the differential 
# expression analysis.
pheno_matrix_subset <- pheno_matrix[toSelect, ]
ge_matrix_subset <- ge_matrix[ , toSelect]

dim(pheno_matrix_subset)
dim(ge_matrix_subset)

In [ ]:
# Now we have to generate the DESeq2 dataset using the phenotype matrix subset (ge_matrix_subset) 
# and count matrix subset (pheno_matrix_subset) that we have created above. Furthermore, we need to
# set the design formula to measure the effect of the variable "cytokine_condition".
# Remember that we have selected the samples to have either cytokine_condition, i.e. phenotype
# Th0 or Th2, by setting the design to column cytokine_condition we set our DESeq up to measure 
# the differtially expressed genes (DEGs) between these two phenotypes.

# dds <- DESeqDataSetFromMatrix(...)
dds <- DESeqDataSetFromMatrix(countData = ge_matrix_subset,
                              colData = pheno_matrix_subset,
                              design = ~ cytokine_condition)

# apply minimal filtering, keeping only genes (rows) with at least 10 counts between all retained samples
keep <- rowSums(counts(dds)) >= 10
dds <- dds[keep,]
dds

In [ ]:
# This DESeq2 dataset can now be used to plot a principle component analysis,
# e.g. to inspect whether the samples separate according to the applied treatment.

plotPCA(rlog(dds), intgroup = 'cytokine_condition')

In [ ]:
# However, we dont want to see a separation of samples by by sequencing batch as this would
# constitute a confounding factor. Let's check that

plotPCA(rlog(dds), intgroup = 'sequencing_batch')

In [ ]:
# Now we can run the differential expression analysis with the DESeq command
dds <- DESeq(object = dds)
# The result of the analysis is then accessible via the "results" command:
DESeqRes <- results(dds)

# We can also discard genes with adjusted P-values of NA, which DESeq2 assignes when certain quality control
# criteria are not met, such as low counts
DESeqRes <- DESeqRes[!is.na(DESeqRes$padj),]

In [ ]:
# --- 1. Load Data ---
# Define paths and load the gene expression and phenotype matrices.
ge_matrix_path <- "./data/NCOMMS-19-7936188_bulk_RNAseq_raw_counts.txt.gz"
pheno_matrix_path <- "./data/NCOMMS-19-7936188_bulk_RNAseq_metadata.txt.gz"
ge_matrix <- read.table(ge_matrix_path, header = TRUE, row.names = 1)
pheno_matrix <- read.table(pheno_matrix_path, header = TRUE, row.names = "sample_id")

# --- 2. Define the Analysis Function ---
# This function must be in your R environment before you can call it.
perform_deg_analysis <- function(cell_type_name, full_pheno_matrix, full_ge_matrix) {
  to_select <- which(full_pheno_matrix$stimulation_time == "5d" &
                     full_pheno_matrix$cytokine_condition %in% c('Th2', 'Th0') &
                     full_pheno_matrix$cell_type == cell_type_name)
  if (length(to_select) == 0) {
    warning(paste("No samples found for cell type:", cell_type_name))
    return(NULL)
  }
  pheno_subset <- full_pheno_matrix[to_select, ]
  ge_subset <- full_ge_matrix[, rownames(pheno_subset)]
  if (!all(colnames(ge_subset) == rownames(pheno_subset))) {
      stop("Mismatch between count matrix columns and phenotype rows.")
  }
  ge_subset <- as.matrix(ge_subset)
  mode(ge_subset) <- "integer"
  if (any(is.na(ge_subset))) {
      stop("Non-numeric values found in count data after coercion to integer.")
  }
  dds <- DESeqDataSetFromMatrix(countData = ge_subset,
                                colData = pheno_subset,
                                design = ~ cytokine_condition)
  keep <- rowSums(counts(dds)) >= 10
  dds <- dds[keep, ]
  dds <- DESeq(dds)
  res <- results(dds)
  res <- res[!is.na(res$padj), ]
  res_sorted <- res[order(res$padj), ]
  return(list(dds = dds, results = res_sorted))
}

# --- 3. Run the Analysis ---
# This is the crucial step that creates the 'naive_analysis' object.
naive_analysis <- perform_deg_analysis("CD4_Naive", pheno_matrix, ge_matrix)

# Now your inspection code will work!
print("Analysis complete. You can now inspect the 'naive_analysis' object.")


In [ ]:
# Now we can filter out the statistically significant differentially expressed genes.
# Apply two criteria to create an index, column padj <= 0.01 and the 
# absolute value of column log2FoldChange > 1, hint: use the function abs

# The 'idx' variable should store the row numbers of the genes that meet both criteria.
# We use `which()` to get the indices of rows where the conditions are TRUE.
# 1. `DESeqResSorted$padj <= 0.01`: Filters for genes with an adjusted p-value of 0.01 or less.
# 2. `abs(DESeqResSorted$log2FoldChange) > 1`: Filters for genes with a log2 fold change greater than 1 or less than -1.
# The `&` operator combines these two conditions, so only genes satisfying both are selected.
idx <- which(DESeqResSorted$padj <= 0.01 & abs(DESeqResSorted$log2FoldChange) > 1)

# The line above completes the "your code here" section.

cat("number of significant DEGs:" , length(idx), "\n")

# And our 10 genes with the stronges response are:
# This line uses the `idx` to subset the sorted results, and `head()` gets the first 10.
DESeqResSorted[head(idx, 10),]



In [ ]:
# Finally, a volcano plot is a good way of visualizing the result of the differential expression analysis,
# including the log-fold-change and P-value of the considered genes.

EnhancedVolcano(DESeqRes, 
                lab = rownames(DESeqRes), 
                x = 'log2FoldChange', y = 'padj', 
                subtitle = 'Th2 vs Th0', 
                labSize = 3, 
                pCutoff = 0.01,
                FCcutoff = 1,
                drawConnectors = FALSE)

## Now we repeat the DEG analysis for memory cells

In [ ]:
# Again, we have to create an index "toSelect", selecting samples (rows) from the phenotype 
# matrix "pheno_matrix" where column "stimulation_time" equals "5d", the cell phenotype 
# ("cytokine_condition") equals either 'Th2' or'Th0', and the sample cell type "cell_type" is "CD4_Memory"

# toSelect <-

# Again, we have to create an index "toSelect", selecting samples (rows) from the phenotype 
# matrix "pheno_matrix" where column "stimulation_time" equals "5d", the cell phenotype 
# ("cytokine_condition") equals either 'Th2' or'Th0', and the sample cell type "cell_type" is "CD4_Memory"

toSelect <- pheno_matrix$stimulation_time == "5d" &
            pheno_matrix$cytokine_condition %in% c('Th2', 'Th0') &
            pheno_matrix$cell_type == "CD4_Memory"


# We use this index now to subset our samples before doing the differential 
# expression analysis.
pheno_matrix_subset <- pheno_matrix[toSelect, ]
ge_matrix_subset <- ge_matrix[ , toSelect]

dim(pheno_matrix_subset)
dim(ge_matrix_subset)

# We use this index now to subset our samples before doing the differential 
# expression analysis.
pheno_matrix_subset <- pheno_matrix[toSelect, ]
ge_matrix_subset <- ge_matrix[ , toSelect]

dim(pheno_matrix_subset)
dim(ge_matrix_subset)

In [ ]:
# Run this line to see the available condition names
unique(pheno_matrix_subset$cytokine_condition) 




In [ ]:
# Generate the DESeq2 dataset using the phenotype matrix subset and count matrix subset. The 
# design formula is still "cytokine_condition".

# ddsMem <- DESeqDataSetFromMatrix(...)
# --- 3. Perform DEG Analysis for Naive and Memory Cells ---
print("Performing DEG analysis for Naive CD4+ cells...")
naive_analysis <- perform_deg_analysis("CD4_Naive", pheno_matrix, ge_matrix)

print("Performing DEG analysis for Memory CD4+ cells...")
memory_analysis <- perform_deg_analysis("CD4_Memory", pheno_matrix, ge_matrix)

# It's a good practice to ensure the column names of the count matrix match the
# row names of the phenotype matrix.
if (!all(colnames(ge_matrix_subset) == rownames(pheno_matrix_subset))) {
    # This reorders the columns of the count matrix to match the phenotype data
    ge_matrix_subset <- ge_matrix_subset[, rownames(pheno_matrix_subset)]
}

# The key step: remove unused factor levels from the subsetted phenotype data.
pheno_matrix_subset <- droplevels(pheno_matrix_subset)

# Now, create the DESeqDataSet object with the cleaned data.
ddsMem <- DESeqDataSetFromMatrix(countData = ge_matrix_subset,
                                 colData = pheno_matrix_subset,
                                 design = ~ cytokine_condition)

# apply minimal filtering, keeping only genes (rows) with at least 10 counts between all retained samples
keep <- rowSums(counts(ddsMem)) >= 10
ddsMem <- ddsMem[keep,]
ddsMem


In [ ]:
# This DESeq2 dataset can now be used to plot a principle component analysis,
# e.g. to inspect whether the samples separate according to the applied treatment.

plotPCA(rlog(ddsMem), intgroup = 'cytokine_condition')

In [ ]:
# However, we dont want to see a separation of samples by by sequencing batch as this would
# constitute a confounding factor. Let's check that

plotPCA(rlog(ddsMem), intgroup = 'sequencing_batch')

In [ ]:
# Now we can run the differential expression analysis with the DESeq command
ddsMem <- DESeq(object = ddsMem)
# The result of the analysis is then accessible via the "results" command:
DESeqMemRes <- results(ddsMem)

# We can also discard genes with adjusted P-values of NA, which DESeq2 assignes when certain quality control
# criteria are not met, such as low counts
DESeqMemRes <- DESeqMemRes[!is.na(DESeqMemRes$padj),]

In [ ]:
# ... (previous code to generate ddsMem) ...

# Now we can run the differential expression analysis with the DESeq command
ddsMem <- DESeq(object = ddsMem)

# The result of the analysis is then accessible via the "results" command:
DESeqMemRes <- results(ddsMem)

# We can also discard genes with adjusted P-values of NA
DESeqMemRes <- DESeqMemRes[!is.na(DESeqMemRes$padj),]

# Now you can inspect the result object. Order the results table by adjusted p-value (column "padj") in
# ascending order, use the function "order"
DESeqMemResSorted <- DESeqMemRes[order(DESeqMemRes$padj), ]

# Inspect the top genes
head(DESeqMemResSorted)

# Now we can filter out the statistically significant differentially expressed genes.
idx <- which(DESeqMemResSorted$padj <= 0.01 & abs(DESeqMemResSorted$log2FoldChange) > 1)

cat("number of significant DEGs:" , length(idx), "\n")

# And our 10 genes with the strongest response are:
head(DESeqMemResSorted[idx, ], 10)

# ... (rest of your analysis and plotting code) ...


In [ ]:
# Now we can filter out the statistically significant differentially expressed genes.
# Apply two criteria to create an index, column padj <= 0.01 and the 
# absolute value of column log2FoldChange > 1, hint: use the function abs

# idx <- ...

idx <- which(DESeqMemResSorted$padj <= 0.01 & abs(DESeqMemResSorted$log2FoldChange) > 1)


cat("number of significant DEGs:" , length(idx), "\n")

# And our 10 genes with the stronges response are:
DESeqMemResSorted[head(idx, 10),]

In [ ]:
# Finally, a volcano plot is a good way of visualizing the result of the differential expression analysis,
# including the log-fold-change and P-value of the considered genes.

EnhancedVolcano(DESeqMemRes, 
                lab = rownames(DESeqMemRes), 
                x = 'log2FoldChange', y = 'padj', 
                subtitle = 'Memory cells - Th2 vs Th0', 
                labSize = 3, 
                pCutoff = 0.01,
                FCcutoff = 1,
                drawConnectors = FALSE)

## Compare lists of DEGs to find shared and unique genes between naive and memory CD4+ cells

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("apeglm")


In [ ]:
#
# This script performs a differential gene expression (DEG) analysis on bulk RNA-seq data
# from the paper "Single-cell transcriptomics identifies an effectorness gradient shaping 
# the response of CD4+ T cells to cytokines" by Cano-Gamez et al.
#

# --- 0. Load Required Packages ---
# Ensure these packages are installed. If not, run:
# if (!requireNamespace("BiocManager", quietly = TRUE)) install.packages("BiocManager")
# BiocManager::install(c("ggplot2", "DESeq2", "EnhancedVolcano"))

require(ggplot2, quietly = TRUE)
require(DESeq2, quietly = TRUE)
require(EnhancedVolcano, quietly = TRUE)

# --- 1. Define the Analysis Function ---
# This function encapsulates the DESeq2 differential expression analysis pipeline.
perform_deg_analysis <- function(cell_type_name, full_pheno_matrix, full_ge_matrix) {
  
  # Create an index to select relevant samples
  to_select <- which(full_pheno_matrix$stimulation_time == "5d" &
                     full_pheno_matrix$cytokine_condition %in% c('Th2', 'Th0') &
                     full_pheno_matrix$cell_type == cell_type_name)
  
  # Check if any samples were found
  if (length(to_select) == 0) {
    warning(paste("No samples found for cell type:", cell_type_name))
    return(NULL)
  }
  
  # Subset the phenotype and gene expression matrices
  pheno_subset <- full_pheno_matrix[to_select, ]
  ge_subset <- full_ge_matrix[, rownames(pheno_subset)]
  
  # Crucial: Drop unused factor levels from the subsetted phenotype data
  pheno_subset <- droplevels(pheno_subset) 
  
  # Ensure column names of countData match row names of colData
  if (!all(colnames(ge_subset) == rownames(pheno_subset))) {
      stop("Column names of countData do not match row names of colData after subsetting.")
  }
  
  # Create the DESeqDataSet object
  dds <- DESeqDataSetFromMatrix(countData = ge_subset,
                                colData = pheno_subset,
                                design = ~ cytokine_condition)
  
  # Apply minimal filtering
  keep <- rowSums(counts(dds)) >= 10
  dds <- dds[keep, ]
  
  # Handle cases where no genes pass filtering
  if (nrow(dds) == 0) {
    warning(paste("No genes passed filtering for cell type:", cell_type_name))
    return(NULL)
  }

  # Run the DESeq analysis
  dds <- DESeq(dds)
  res <- results(dds)
  
  # Discard genes with NA adjusted P-values
  res <- res[!is.na(res$padj),] 
  
  # Handle cases where no significant results remain after NA filtering
  if (nrow(res) == 0) {
    warning(paste("No results with valid p-values for cell type:", cell_type_name))
    return(NULL)
  }

  # Sort results by adjusted p-value
  res_sorted <- res[order(res$padj), ]
  return(list(dds = dds, results = res_sorted))
}

# --- 2. Load and Prepare Data ---
ge_matrix_path <- "./data/NCOMMS-19-7936188_bulk_RNAseq_raw_counts.txt.gz"
pheno_matrix_path <- "./data/NCOMMS-19-7936188_bulk_RNAseq_metadata.txt.gz"

ge_matrix <- read.table(ge_matrix_path, header = TRUE, row.names = 1)
pheno_matrix <- read.table(pheno_matrix_path, header = TRUE)

# Set row names and then remove the now-redundant 'sample_id' column
rownames(pheno_matrix) <- pheno_matrix$sample_id
pheno_matrix$sample_id <- NULL

print("Data loaded and prepared successfully.")

# --- 3. Perform DEG Analysis for Naive and Memory Cells ---
print("Performing DEG analysis for Naive CD4+ cells...")
naive_analysis <- perform_deg_analysis("CD4_Naive", pheno_matrix, ge_matrix)

print("Performing DEG analysis for Memory CD4+ cells...")
memory_analysis <- perform_deg_analysis("CD4_Memory", pheno_matrix, ge_matrix)

# --- 4. Compare and Summarize Results ---
if (!is.null(naive_analysis) && !is.null(memory_analysis)) {
  degs_naive <- rownames(naive_analysis$results[which(naive_analysis$results$padj <= 0.01 & abs(naive_analysis$results$log2FoldChange) > 1), ])
  degs_mem <- rownames(memory_analysis$results[which(memory_analysis$results$padj <= 0.01 & abs(memory_analysis$results$log2FoldChange) > 1), ])
  
  sharedDEGs <- intersect(degs_naive, degs_mem)
  naiveSpecificDEGs <- setdiff(degs_naive, degs_mem)
  memSpecificDEGs <- setdiff(degs_mem, degs_naive)
  
  cat("\n--- Comparison of DEGs ---\n")
  cat("Number of DEGs specific to naive cells:", length(naiveSpecificDEGs), "\n")
  cat("Number of DEGs specific to memory cells:", length(memSpecificDEGs), "\n")
  cat("Number of shared DEGs:", length(sharedDEGs), "\n")
} else {
  print("Cannot compare DEGs: one or both analyses failed.")
}

# --- 5. Visualize Results (Optional) ---
# You can now create plots using the 'naive_analysis' and 'memory_analysis' objects.
# For example, to create a volcano plot for naive cells:
#
# if (!is.null(naive_analysis)) {
#   print(EnhancedVolcano(naive_analysis$results, 
#                         lab = rownames(naive_analysis$results), 
#                         x = 'log2FoldChange', y = 'padj', 
#                         subtitle = 'Naive - Th2 vs Th0', 
#                         pCutoff = 0.01, FCcutoff = 1))
# }



[1] "Data loaded and prepared successfully."
[1] "Performing DEG analysis for Naive CD4+ cells..."


estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Performing DEG analysis for Memory CD4+ cells..."


estimating size factors

estimating dispersions

gene-wise dispersion estimates



In [16]:
# Compare lists of DEGs to find shared and unique genes between naive and memory CD4+ cells
# This step assumes that 'naive_analysis' and 'memory_analysis' objects from the previous analyses exist.

# 1. Get the names of significant DEGs for naive cells from the 'naive_analysis' object.
# We filter for an adjusted p-value (padj) of 0.01 or less and a log2 fold change
# with an absolute value greater than 1.
degs_naive <- rownames(naive_analysis$results[which(naive_analysis$results$padj <= 0.01 & abs(naive_analysis$results$log2FoldChange) > 1), ])

# 2. Get the names of significant DEGs for memory cells from the 'memory_analysis' object.
degs_mem <- rownames(memory_analysis$results[which(memory_analysis$results$padj <= 0.01 & abs(memory_analysis$results$log2FoldChange) > 1), ])

# 3. Find the intersection of the two lists to identify shared DEGs.
# The intersect() function returns a vector of elements that are present in both input vectors.
sharedDEGs <- intersect(degs_naive, degs_mem)

# 4. Find the genes that are unique to the naive cell analysis.
# The setdiff(A, B) function returns elements that are in A but not in B.
naiveSpecificDEGs <- setdiff(degs_naive, degs_mem)

# 5. Find the genes that are unique to the memory cell analysis.
memSpecificDEGs <- setdiff(degs_mem, degs_naive)

# Print a summary of the comparison to the console.
cat("Number of shared DEGs between memory and naive CD4+ cells:", length(sharedDEGs), "\n")
cat("Number of DEGs specific to naive CD4+ cells:", length(naiveSpecificDEGs), "\n")
cat("Number of DEGs specific to memory CD4+ cells:", length(memSpecificDEGs), "\n")


Number of shared DEGs between memory and naive CD4+ cells: 21 
Number of DEGs specific to naive CD4+ cells: 918 
Number of DEGs specific to memory CD4+ cells: 4 
